# ResNet-18
Credit to: [Using ResNet for MNIST in PyTorch 1.7](https://zablo.net/blog/post/pytorch-resnet-mnist-jupyter-notebook-2021/)

##Python Setup

In [ ]:
!pip install torch torchvision
!pip install matplotlib
!pip install import-ipynb
!pip install pytorch_extras
!pip install pytorch_lightning

In [ ]:
from google.colab import drive
drive.mount("mnt")
%cd "mnt/My Drive"

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-7ebb92d86321>", line 2, in <module>
    drive.mount("mnt")
  File "/usr/local/lib/python3.7/dist-packages/google/colab/drive.py", line 254, in mount
    raise ValueError('mount failed' + extra_reason)
ValueError: mount failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'ValueError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinne

ValueError: ignored

In [ ]:
from torchvision.models import resnet18
from torch import nn
import torch
import numpy as np
import import_ipynb
import load_DrawData_with_transform as loader
import pytorch_lightning as pl
from pytorch_lightning.core.decorators import auto_move_data

## Model

In [ ]:
class ResNet(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.model = resnet18()
    self.model.conv1 = nn.Conv2d(1, 32, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    self.loss = nn.CrossEntropyLoss()

  @auto_move_data
  def forward(self, x):
    x = x.unsqueeze(1)
    x = char(x)
    return self.model(x)
  
  def training_step(self, batch, batch_no):
    # implement single training step
    x, y = batch
    logits = self(x)
    loss = self.loss(logits, y)
    return loss
  
  def configure_optimizers(self):
    return torch.optim.RMSprop(self.parameters(), lr=0.005)

## Train

In [ ]:
train_loader, test_loader = loader.train_loader, loader.val_loader

model = ResNet()
model = model.cuda()

trainer = pl.Trainer(
    max_epochs=100,
    progress_bar_refresh_rate=30 # set to >= 20 if running in Google Colab
)

trainer.fit(model, train_loader)

#trainer.save_checkpoint("resnet18_quickdraw.pt")